1) Imports and Config

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict, List
import csv
import os
import uvicorn
import nest_asyncio
from threading import Thread
import ipytest
import httpx

CSV_FILE = "patients.csv"
BASE_URL = "http://127.0.0.1:8000"
ipytest.autoconfig()

2. Data Model

In [2]:
class Patient(BaseModel):
    patient_id: str
    age: int
    sex: str
    BMI: float
    diagnosis_code: str
    sample_taken: bool
    blood_pressure: str
    cholesterol_level: float

3. CSV Helper Functions

In [3]:
def load_patients_from_csv() -> Dict[str, Patient]:
    if not os.path.exists(CSV_FILE):
        return {}
    with open(CSV_FILE, newline='') as f:
        reader = csv.DictReader(f)
        data = {}
        for row in reader:
            row['age'] = int(row['age'])
            row['BMI'] = float(row['BMI'])
            row['sample_taken'] = row['sample_taken'].lower() == "true"
            row['cholesterol_level'] = float(row['cholesterol_level'])
            data[row['patient_id']] = Patient(**row)
        return data

def save_patients_to_csv():
    with open(CSV_FILE, mode='w', newline='', encoding='utf-8') as f:
        fieldnames = list(Patient.__fields__)  # Pydantic v1 compatibility
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for patient in patients.values():
            writer.writerow(patient.dict())
        print("Saving to:", os.path.abspath(CSV_FILE))

4. FastAPI Initialisation

In [4]:
app = FastAPI()
patients: Dict[str, Patient] = load_patients_from_csv()

5. API Endpoints

In [5]:
@app.get("/patients", response_model=List[Patient])
def get_all_patients():
    return list(patients.values())

@app.get("/patients/{patient_id}", response_model=Patient)
def get_patient(patient_id: str):
    if patient_id not in patients:
        raise HTTPException(status_code=404, detail="Patient not found")
    return patients[patient_id]

@app.post("/patients", response_model=Patient, status_code=201)
def create_patient(patient: Patient):
    if patient.patient_id in patients:
        raise HTTPException(status_code=400, detail="Patient already exists")
    patients[patient.patient_id] = patient
    save_patients_to_csv()
    return patient

@app.put("/patients/{patient_id}", response_model=Patient)
def update_patient(patient_id: str, updated: Patient):
    if patient_id not in patients:
        raise HTTPException(status_code=404, detail="Patient not found")
    patients[patient_id] = updated
    save_patients_to_csv()
    return updated

@app.delete("/patients/{patient_id}", status_code=204)
def delete_patient(patient_id: str):
    if patient_id not in patients:
        raise HTTPException(status_code=404, detail="Patient not found")
    del patients[patient_id]
    save_patients_to_csv()

6. Launch Server

In [6]:
nest_asyncio.apply()

def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

Thread(target=run, daemon=True).start()

INFO:     Started server process [20496]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50538 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50540 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50538 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50538 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50547 - "GET /patients HTTP/1.1" 200 OK


7. Automation Tests

In [14]:
# Test 1: Get all patients
def test_get_all_patients():
    response = httpx.get(f"{BASE_URL}/patients")
    assert response.status_code == 200
    assert isinstance(response.json(), list)

# Test 2: Get a valid patient by ID
def test_get_valid_patient(patient_id: str):
    response = httpx.get(f"{BASE_URL}/patients/{patient_id}")
    assert response.status_code == 200
    assert response.json()["patient_id"] == patient_id

# Test 3: Get an invalid (nonexistent) patient
def test_get_invalid_patient(patient_id: str):
    response = httpx.get(f"{BASE_URL}/patients/{patient_id}")
    assert response.status_code == 404

# Test 4: Create a new patient
def test_create_new_patient(patient_id: str):
    new_patient = {
        "patient_id": patient_id,
        "age": 55,
        "sex": "Male",
        "BMI": 28.5,
        "diagnosis_code": "I90",
        "sample_taken": True,
        "blood_pressure": "125/85",
        "cholesterol_level": 5.5
    }
    response = httpx.post(f"{BASE_URL}/patients", json=new_patient)
    assert response.status_code == 201
    assert response.json()["patient_id"] == patient_id

# Test 5: Update an existing patient
def test_update_patient(patient_id: str, new_age: int):
    updated_data = {
        "patient_id": patient_id,
        "age": new_age,
        "sex": "Male",
        "BMI": 29.0,
        "diagnosis_code": "I91",
        "sample_taken": False,
        "blood_pressure": "120/80",
        "cholesterol_level": 5.0
    }
    response = httpx.put(f"{BASE_URL}/patients/{patient_id}", json=updated_data)
    assert response.status_code == 200
    assert response.json()["age"] == new_age

# Test 6: Delete a patient
def test_delete_patient(patient_id: str):
    response = httpx.delete(f"{BASE_URL}/patients/{patient_id}")
    assert response.status_code == 204
    # Confirm deletion
    check = httpx.get(f"{BASE_URL}/patients/{patient_id}")
    assert check.status_code == 404

In [16]:
test_get_all_patients()
test_get_valid_patient("PAT1002")
test_get_invalid_patient("PAT10000")
test_create_new_patient("TEST001")
test_update_patient("TEST001", 50)
test_delete_patient("TEST001")

INFO:     127.0.0.1:50752 - "GET /patients HTTP/1.1" 200 OK
INFO:     127.0.0.1:50753 - "GET /patients/PAT1002 HTTP/1.1" 200 OK
INFO:     127.0.0.1:50754 - "GET /patients/PAT10000 HTTP/1.1" 404 Not Found
Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:50755 - "POST /patients HTTP/1.1" 201 Created
Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:50756 - "PUT /patients/TEST001 HTTP/1.1" 200 OK
Saving to: C:\Users\mfbx2jrj\UKBiobank-Interview\patients.csv
INFO:     127.0.0.1:50757 - "DELETE /patients/TEST001 HTTP/1.1" 204 No Content
INFO:     127.0.0.1:50758 - "GET /patients/TEST001 HTTP/1.1" 404 Not Found
